In [1]:
import ezdxf
from ezdxf.addons.drawing import matplotlib
from ezdxf.addons.drawing import RenderContext, Frontend
from ezdxf.addons.drawing.matplotlib import MatplotlibBackend
import matplotlib.pyplot as plt
import numpy as np
from lens import *

In [2]:
def add_line(msp, p1, p2):
    new_line = msp.add_line(p1, p2, dxfattribs={'layer':'structure'})
    return new_line

def add_arc(msp, vertex_p, R, D):
    center_p = (vertex_p[0]+R, vertex_p[1])
    h = D/2
    start_ang = 0
    end_ang   = 0
    if R > 0:
        start_ang = 180 - np.degrees( np.arcsin(h/R) )
        end_ang   = 180 + np.degrees( np.arcsin(h/R) )
    else:
        start_ang = np.degrees( np.arcsin(h/R) )
        end_ang   =  - start_ang
        
    new_arc = msp.add_arc(center=center_p, radius=abs(R), start_angle=start_ang, end_angle=end_ang, dxfattribs={'layer':'structure'})
    return new_arc

def add_dim(msp, base, p1, p2):
    dim = msp.add_linear_dim(base=base, p1=p1, p2=p2, dimstyle="Standard")
    return dim

In [9]:

def draw_lens(msp, singlet, pos):
    print("Draw lens")
    cd1 = singlet.left_surface.clear_diameter
    cd2 = singlet.right_surface.clear_diameter
    ed1 = singlet.left_surface.edge_diameter
    ed2 = singlet.right_surface.edge_diameter
    D = max(ed1, ed2)
    
    r1 = singlet.left_surface.radius
    r2 = singlet.right_surface.radius
    t  = singlet.thickness
    
    # draw curved surface
    left_pos = pos
    add_arc(msp, left_pos, r1, cd1)
    right_pos = (pos[0]+t, pos[1] )
    add_arc(msp, right_pos, r2, cd2)
    
    # draw flat surface
    pt1 = (left_pos[0]+singlet.left_surface.sag(cd1/2), left_pos[1]+cd1/2)
    pt2 = (left_pos[0]+singlet.left_surface.sag(cd1/2), left_pos[1]+D/2)
    add_line(msp, pt1, pt2)
    
    pt1 = (left_pos[0]+singlet.left_surface.sag(cd1/2), left_pos[1]-cd1/2)
    pt2 = (left_pos[0]+singlet.left_surface.sag(cd1/2), left_pos[1]-D/2)
    add_line(msp, pt1, pt2)
    
    pt1 = (right_pos[0]+singlet.right_surface.sag(cd2/2), left_pos[1]+cd2/2)
    pt2 = (right_pos[0]+singlet.right_surface.sag(cd2/2), left_pos[1]+D/2)
    add_line(msp, pt1, pt2)
    
    pt1 = (right_pos[0]+singlet.right_surface.sag(cd2/2), left_pos[1]-cd2/2)
    pt2 = (right_pos[0]+singlet.right_surface.sag(cd2/2), left_pos[1]-D/2)
    add_line(msp, pt1, pt2)
    
    # draw koba
    pt1 = (left_pos[0]+singlet.left_surface.sag(cd1/2), left_pos[1]+D/2)
    pt2 = (right_pos[0]+singlet.right_surface.sag(cd2/2), right_pos[1]+D/2)
    add_line(msp, pt1, pt2)
    
    pt1 = (left_pos[0]+singlet.left_surface.sag(cd1/2), left_pos[1]-D/2)
    pt2 = (right_pos[0]+singlet.right_surface.sag(cd2/2), right_pos[1]-D/2)
    add_line(msp, pt1, pt2)
    
    # draw dimensions
    # center thickness
    pt1 = pos
    pt2 = (pos[0]+t, pos[1])
    base = (pos[0]+t/2, pos[1] - (D/2)*1.)
    add_dim(msp,base, pt1, pt2)
    

In [7]:

doc = ezdxf.readfile("./SolidEdge/schematic_A4.dxf")
msp = doc.modelspace()

layers = ["structure", "dimension", "text", "frame"] #構造物 寸法 文字列 図枠
for layer in layers:
    doc.layers.add(name=layer)#特に属性は指定しない 

lens1 = Singlet(40.0, 12, 15, -40.00, 12, 15, 'N-BK7', 5) 
draw_lens(msp, lens1, (120,130))

lens2 = Singlet(-10.0, 7, 15, 15.00, 15, 15, 'N-BK7', 2) 
draw_lens(msp, lens2, (150,130))

lens3 = Singlet(-10.0, 7, 15, -55.00, 15, 15, 'N-BK7', 3) 
draw_lens(msp, lens3, (200,130))

doc.saveas("lens.dxf")



Draw lens
Draw lens
Draw lens
